In [ ]:
#Last Edited: 2025/05/06
#File changed more recently (2025/11/18) to commit (with Cell outputs deleted)

In [ ]:
#This is a follow up to "split_h5_to_24hour_add_annotations.ipynb"

#I'm using the "science_env" kernel

#I've created 2 python files:
# F:\Documents\GitHub\ml_development\ADCP_ML\convert_monthly_mat_to_h5.py
#F:\Documents\GitHub\ml_development\ADCP_ML\split_h5_to_24hr_files.py

#I'll want to create a loop that scans the monthly mat folder structure and outputs hourly files
# There are also intermediate monthly h5 files, so will probably be good to clean those up as this runs

In [1]:
#Code to test folder scan

import os
import datetime

data_parent = r'F:\Documents\Projects\ADCP\scan_for_data\BACAX\ADCP2MHZ\\'
folder_list = os.listdir(data_parent)

#I only want folders including and after '20110715', as device is oriented down-facing prior to this date
cutoff_date = datetime.datetime(2011,7,15)
for folder in folder_list:
    print(folder)
    folder_date = datetime.datetime.strptime(folder,'%Y%m%d')
    if folder_date >= cutoff_date:
        file_list = os.listdir(data_parent + folder)
        mat_files = {k for k in file_list if os.path.splitext(k)[1] == ".mat"}
        print(mat_files)


20100517
20100601
20100701
20100801
20100901
20101001
20101101
20101201
20110101
20110201
20110301
20110401
20110501
20110601
20110701
20110715
{'BarkleyCanyon_BarkleyCanyonAxis_AcousticDopplerCurrentProfiler2MHz_20110715T195000Z_20110720T172000Z-Ensemble300s_binMapNearest.mat', 'BarkleyCanyon_BarkleyCanyonAxis_AcousticDopplerCurrentProfiler2MHz_20110728T210500Z_20110801T000000Z-Ensemble300s_binMapNearest.mat', 'BarkleyCanyon_BarkleyCanyonAxis_AcousticDopplerCurrentProfiler2MHz_20110720T172000Z_20110728T210500Z-Ensemble300s_binMapNearest.mat'}
20110801
{'BarkleyCanyon_BarkleyCanyonAxis_AcousticDopplerCurrentProfiler2MHz_20110801T000000Z_20110901T000000Z-Ensemble300s_binMapNearest.mat'}
20110901
{'BarkleyCanyon_BarkleyCanyonAxis_AcousticDopplerCurrentProfiler2MHz_20110901T000000Z_20111001T000000Z-Ensemble300s_binMapNearest.mat'}
20111001
{'BarkleyCanyon_BarkleyCanyonAxis_AcousticDopplerCurrentProfiler2MHz_20111001T000000Z_20111101T000000Z-Ensemble300s_binMapNearest.mat'}
20111101
{'Bark

In [2]:
import os
import datetime

# Add repo root to Python path - Needed to import from src folder
import sys
from pathlib import Path
repo_root = Path().resolve().parent  # notebooks/ → ADCP-CNN-QAQC
sys.path.append(str(repo_root))

from src import convert_monthly_mat_to_h5
from src import split_h5_to_24hr_files

######################################
# Convert mat to h5 (still monthly format)
######################################

data_parent = r'F:\Documents\Projects\ADCP\scan_for_data\BACAX\ADCP2MHZ\\'
folder_list = os.listdir(data_parent)

# Define output folder -  Monthly h5
h5_monthly_folder = r'F:\Documents\Projects\ML\ADCP_ML\h5_files\\'

#Output folder - 24hr h5:
h5_24h_folder = r'F:\Documents\Projects\ML\ADCP_ML\h5_24h_files\\' 

#.mat format annotations file:
annotations_file = r'F:\Documents\Projects\ML\ADCP_ML\annotations_table_ed05_revised.mat'

#I only want folders including and after '20110715', as device is oriented down-facing prior to this date
#I can also adjust this date if the code crashes part way through
cutoff_date = datetime.datetime(2011,7,15)

for folder in folder_list:
    print(folder)
    folder_date = datetime.datetime.strptime(folder,'%Y%m%d')
    if folder_date >= cutoff_date:
        # Path to your .mat file
        file_list = os.listdir(data_parent + folder)
        mat_files = {k for k in file_list if os.path.splitext(k)[1] == ".mat"}
        print(mat_files)

        #Combine the filenames into a proper path:
        mat_paths = []
        for filename in mat_files:
            mat_paths.append(data_parent + folder + '\\' + filename) 

        #Run the extraction
        for mat_path in mat_paths:
            #print(mat_path)
            convert_monthly_mat_to_h5.extract_mat_to_h5(mat_path, h5_monthly_folder) 

        ######################################
        # Split to 24 hours, embed annotations
        # and save the time in python format
        ######################################
            
        #Paths to month(ish) HDF5 source file(s):
        for mat_path in mat_paths:
            filename_mat = os.path.basename(mat_path)
            filename_h5 = os.path.splitext(filename_mat)[0] + '.h5'
            input_file = h5_monthly_folder + filename_h5

            #Output folder:
            # h5_24h_folder = r'F:\Documents\Projects\ML\ADCP_ML\h5_24h_files\\' 

            #.mat format annotations file:
            # annotations_file = r'F:\Documents\Projects\ML\ADCP_ML\annotations_table_ed05_revised.mat'

            print('Splitting files from folder {}'.format(folder))

            split_h5_to_24hr_files.split_h5_to_24hr_files_with_ann(
                input_file,             # your big HDF5 source (created with import_monthly_mat_to_h5.py)
                h5_24h_folder,          # output dir for 24hr files
                annotations_file,        # your .mat annotations file
            )

######################################
#Test code to extract and plot the data 
######################################
#filename = r'F:\\Documents\\GitHub\\ml_development\\ADCP_ML\\h5_24h_files\\20240406T000000_20240406T235959.h5'
#make_sanity_plots(filename, outdir="./h5_24hr_figs", show=True)

20100517
20100601
20100701
20100801
20100901
20101001
20101101
20101201
20110101
20110201
20110301
20110401
20110501
20110601
20110701
20110715
{'BarkleyCanyon_BarkleyCanyonAxis_AcousticDopplerCurrentProfiler2MHz_20110715T195000Z_20110720T172000Z-Ensemble300s_binMapNearest.mat', 'BarkleyCanyon_BarkleyCanyonAxis_AcousticDopplerCurrentProfiler2MHz_20110728T210500Z_20110801T000000Z-Ensemble300s_binMapNearest.mat', 'BarkleyCanyon_BarkleyCanyonAxis_AcousticDopplerCurrentProfiler2MHz_20110720T172000Z_20110728T210500Z-Ensemble300s_binMapNearest.mat'}

== META ==
  deviceID: <class 'int'> (11302)
  creationDate: <class 'str'> (20250213T205502Z)
  deviceName: <class 'str'> (Nortek Aquadopp HR-Profiler 2 MHz 2700)
  deviceCode: <class 'str'> (BC_POD1_AD2M)
  deviceCategory: <class 'str'> (Acoustic Doppler Current Profiler 2 MHz)
  deviceCategoryCode: <class 'str'> (ADCP2MHZ)
  lat: <class 'float'> (48.3167116667)
  lon: <class 'float'> (-126.0501283333)
  depth: <class 'float'> (986.58)
  device